In [3]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [4]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [5]:
ps = ps[["Date", "User", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "Notes", "Internal version", "Picogreen", "Conc.", ">400", "Ratio", "PCR cycles", "SM_QC_PF"]]
ps.dropna(subset = ["Post patch?", "Picogreen", "Conc.", ">400"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,Picogreen,Conc.,>400,Ratio,PCR cycles,SM_QC_PF
4968,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,71,19.60,8,0.41,21.0,fail
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,608,854.70,461,0.54,21.0,pass
4970,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,6,1.00,1,1,21.0,fail
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,1536,1370.50,979,0.71,21.0,pass
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,582,834.90,456,0.55,21.0,pass
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,1154,1471.70,1193,0.81,21.0,pass
4974,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,1504,2227.80,1319,0.59,21.0,pass
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,1330,2195.50,1567,0.71,21.0,pass
4978,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,839,1242.00,411,0.33,21.0,fail
4979,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,1292,1461.20,751,0.51,21.0,pass


In [6]:
def get_count(x):    
    count = x["Post patch?"].count()
    return count

def calc_pct(part,total):
    pct = float(get_count(part)) / get_count(total)
    return pct

In [7]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 501))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 500))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 501))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 500))]
    return no
    
def pass_only(df):
    pas = df[df["SM_QC_PF"] == "pass"]
    return pas

In [ ]:
nuc = nucleated(ps)
nuc["post patch class"] = "Nucleated"

part_nuc = partial_nucleated(ps)
part_nuc["post patch class"] = "Partial nucleated"

out_out = outside_out(ps)
out_out["post patch class"] = "Outisde out"

noSeal = no_seal(ps)
noSeal["post patch class"] = "No seal"

entirecell = ps[ps["Po"]]